# Problem Set #2

MACS 30000, Dr. Evans

Haihao Guo

## Exercise 1

### A

In [5]:
import timeit
import warnings
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from statistics import mean
warnings.filterwarnings("ignore")

df1 = pd.read_csv('data/Auto.csv',na_values="?")
df1.dropna(inplace=True)
tf = df1['mpg']>=df1['mpg'].median()
df1['mpg_high'] = np.where(tf,1,0)

df1['orgn1'] = 0
df1.orgn1[df1['origin']==1] = 1
df1['orgn2'] = 0
df1.orgn2[df1['origin']==2] = 1

y = df1['mpg_high'].values
X = df1[['cylinders', 'displacement', 'horsepower', 'weight','acceleration', 'year', 'orgn1', 'orgn2']].values
start_time = timeit.default_timer()
mse = []
for i in range(100):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.35, random_state=i+100)
    LogReg = LogisticRegression().fit(X_train, y_train)
    y_pred = LogReg.predict(X_test)
    mse_now = (y_test == y_pred).mean()
    mse.append(mse_now)
mse_avg = mean(mse)
print('The average mse is', mse_avg)
print('This computation takes',timeit.default_timer() - start_time,'seconds')


The average mse is 0.8986231884057971
This computation takes 1.3695818989999964 seconds


### B

In [2]:
import multiprocessing
num_cores = multiprocessing.cpu_count()
print('Number of available cores is', num_cores)

Number of available cores is 8


In [7]:
import multiprocessing
from dask import compute, delayed
import dask.multiprocessing
import time
from statistics import mean
num_cores = multiprocessing.cpu_count()
print('Number of available cores is', num_cores)

def doublecheck(bootstrap, seed, Data):
    X, y = Data
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.35, random_state=seed+bootstrap)
    LogReg = LogisticRegression()
    LogReg.fit(X_train, y_train)
    y_pred = LogReg.predict(X_test)
    err_rate_now = (y_test == y_pred).mean()
    return err_rate_now
time_start2 = time.clock()
err_rate_2 = []
for i in range(100):
    err_rate_2.append(delayed(doublecheck)(i,100,[X,y]))
err_rate_2_avg = delayed(mean)(err_rate_2)
err_rate_2_avg = compute(err_rate_2_avg, num_workers=num_cores)[0]
time_end2 = time.clock()
print('average error rate =', err_rate_2_avg)
print('computation time =',time_end2-time_start2,'seconds')

Number of available cores is 8
average error rate = 0.8986231884057971
computation time = 1.827363 seconds


Using Dask to perform parallel takes even longer to finish the task. A potential reason for that is despite the fact that the tasks are distributed across different CPU cores, it spends too much time on the inter-core communication. The imprivement may be obvious if we have a big data here.